# Работа с базами данных (2)

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Лекция "Работа с базами данных"
* https://sqliteonline.com/
* https://docs.python.org/3/library/sqlite3.html
* https://www.sqlitetutorial.net/sqlite-index/
* https://docs.python.org/3/library/sqlite3.html#sqlite3.IntegrityError
* https://www.sqlitetutorial.net/sqlite-alter-table/
* https://www.sqlitetutorial.net/sqlite-create-view/
* https://habr.com/ru/post/664000/
* https://learnsql.com/blog/what-is-common-table-expression/


## Задачи для совместного разбора

In [1]:
import pandas as pd
import sqlite3

# данные
students = pd.DataFrame(
    [
        ("Сотников Евгений Янович", 1),
        ("Степанова Виктория Константиновна", 1),
        ("Горелова Вероника Яновна", 2),
        ("Гришин Иван Романович", 3),
    ],
    columns=["name", "group_id"],
)
groups = list(zip([1, 2, 3], ["ПМ20-1", "ПМ20-2", "ПМ20-3"]))

con = sqlite3.connect("demo.sqlite")
con.execute("PRAGMA foreign_keys = 1")
cur = con.cursor()

# создаем таблицы
sql = """
DROP TABLE IF EXISTS StudentGroup;
DROP TABLE IF EXISTS Student;
CREATE TABLE StudentGroup (
    id int PRIMARY KEY,
    name varchar
);

CREATE TABLE Student(
    name VARCHAR PRIMARY KEY,
    group_id INT,
    FOREIGN KEY (group_id) REFERENCES StudentGroup(id)
);
"""
cur.executescript(sql)
con.commit()

# добавляем записи
sql = """
INSERT INTO StudentGroup(id, name) VALUES (?, ?)
"""
cur.executemany(sql, groups)
con.commit()

students.to_sql("Student", con, if_exists="append", index=False)

4

1\. Добавить столбец Age со значением по умолчанию. Добавить запись к таблицу

In [2]:
sql = """
ALTER TABLE Student
    ADD COLUMN Age INT DEFAULT 18
"""
cur.execute(sql)
con.commit()

In [3]:
pd.read_sql_query('SELECT * FROM Student', con)

,name,group_id,Age
0,Сотников Евгений Янович,1,18
1,Степанова Виктория Константиновна,1,18
2,Горелова Вероника Яновна,2,18
3,Гришин Иван Романович,3,18


In [7]:
sql = """
INSERT INTO Student(name, group_id, Age)
    VALUES("Аств Влда", 3, 20)
"""
try:
    cur.execute(sql)
except sqlite3.OperationalError as e:
    print("OperationalError: ", e)
else:
    con.commit()

2\. Занумеруйте студентов в рамках каждой группы.

In [12]:
pd.read_sql_query("""
    SELECT name, 
        group_id,
        ROW_NUMBER() OVER(PARTITION BY group_id ORDER BY name) as rid
    FROM Student
""", con)

,name,group_id,rid
0,Сотников Евгений Янович,1,1
1,Степанова Виктория Константиновна,1,2
2,Горелова Вероника Яновна,2,1
3,Аств Влда,3,1
4,Гришин Иван Романович,3,2


3\. Выведите уникальные номера студентов

In [13]:
pd.read_sql_query("""
SELECT DISTINCT rid 
    FROM(
    SELECT name, 
        group_id,
        ROW_NUMBER() OVER(PARTITION BY group_id ORDER BY name) as rid
    FROM Student
    )
""", con)

,rid
0,1
1,2


In [14]:
sql = """
CREATE VIEW Student_view AS 
    SELECT name, 
        group_id,
        ROW_NUMBER() OVER(PARTITION BY group_id ORDER BY name) as rid
    FROM Student
"""
cur.execute(sql)
con.commit()

In [15]:
pd.read_sql_query("""
    SELECT DISTINCT rid
    FROM Student_view
""", con)

,rid
0,1
1,2


In [17]:
pd.read_sql_query("""
    WITH StudentCTE AS (
        SELECT name,
            group_id,
            ROW_NUMBER() OVER(PARTITION BY group_id ORDER BY name) as rid
    FROM Student
    )
    SELECT DISTINCT rid
    FROM (StudentCTE)
""", con)

,rid
0,1
1,2


## Лабораторная работа 4

__При решении данных задач не подразумевается использования циклов или генераторов Python в ходе работы с пакетами `numpy` и `pandas`, если в задании не сказано обратного. Решения задач, в которых для обработки массивов `numpy` или структур `pandas` используются явные циклы (без согласования с преподавателем), могут быть признаны некорректными и не засчитаны.__

In [1]:
import pandas as pd
import sqlite3

__Для начала работы подключитесь к БД `recipes.db` и создайте объект-курсор.__

In [2]:
con = sqlite3.connect("recipes.db")

In [3]:
cur = con.cursor()

<p class="task" id="1"></p>

1\. Создайте уникальный индекс для таблицы `Review` для обеспечения уникальности сочетания значений в полях `user_id` и `recipe_id`. 

In [4]:
cur.execute("CREATE UNIQUE INDEX user_recipe ON Review(user_id, recipe_id)");
con.commit()

<p class="task" id="2"></p>

2\. Напишите функцию `add_review(review_id, user_id, recipe_id, date, rating, review)`, которая добавляет запись в таблицу `Review`. В случае успешного добавления функция должна вернуть значение 0. В случае нарушения ограничения целостности функция должна вернуть значение 1. В случае любых других ошибок функция должна вернуть значение 2. Продемонстрируйте работу функции, попытавшись добавить одну и ту же запись дважды в двух ячейках подряд.

Для решения задачи воспользуйтесь механизмом try - except и обработайте соответствующее исключение.

In [15]:
def add_review(review_id, user_id, recipe_id, date, rating, review):
    try:
        sql = """
            INSERT INTO Review(id, user_id, recipe_id, date, rating, review)
                VALUES (?, ?, ?, ?, ?, ?)
        """
        cur.execute(sql, [review_id, user_id, recipe_id, date, rating, review])
        con.commit()
        return 0
    except sqlite3.IntegrityError:
        return 1
    else:
        return 2

In [16]:
add_review(1, 111, 121, '15.03.2023', 5, 'Very nice!')

0

In [17]:
add_review(1, 111, 121, '15.03.2023', 5, 'Very nice!')

1

<p class="task" id="3"></p>

3\. _Измените_ таблицу Review, добавив в нее поле `toxic` булева типа. 

In [19]:
sql = """
ALTER TABLE Review
    ADD COLUMN toxic bool
"""
cur.execute(sql)
con.commit()

<p class="task" id="4"></p>

4\. Вам дан классификатор `clf`, который классифицирует тексты отзывов как токсичные (`True`) и не токсичные (`False`).
Напишите функцию `classify_reviews`, которая итеративно получает пакет (батч) `batch_size` строк из таблицы Reviews, у которых не проставлено значение в столбце `toxic`, делает для них прогноз при помощи модели `clf` и обновляет соответствующие строки в БД. Данная процедура выполняется до тех пор, пока в БД есть строки, для которых требуется получить прогноз.

Продемонстрируйте результат, выведя на экран количество токсичных и не токсичных отзывов в таблице.

In [4]:
from sklearn.dummy import DummyClassifier

clf = DummyClassifier(strategy="uniform").fit(None, [True, False])

In [5]:
def classify_reviews(batch_size=10000):
    df = pd.read_sql_query("""
        SELECT id, toxic 
        FROM Review
        WHERE toxic IS NULL
        LIMIT ?
        """, con, params=[batch_size])
    while df.size > 0:
        df['toxic'] = clf.predict(df['toxic'])
        zp_list = list(zip(df['toxic'], df['id']))
        for i in zp_list:
            cur.execute("""
                UPDATE Review SET toxic = ? WHERE id = ?
            """, i)
        df = pd.read_sql_query("""
            SELECT id, toxic 
            FROM Review
            WHERE toxic IS NULL
            LIMIT ?
            """, con, params=[batch_size])
        con.commit()

classify_reviews()

In [6]:
pd.read_sql_query("""
    SELECT COUNT(review) as "Count of reviews", toxic FROM Review GROUP BY toxic
""", con)

,Count of reviews,toxic
0,63244,0
1,63436,1


<p class="task" id="5"></p>

5\. Создайте представление `RecipeWithYear`, в котором добавлен дополнительный столбец `year`, содержащий год даты из столбца `submitted`. Сделайте выборку из этого представления и выведите на экран количество рецептов с разбивкой по годам.

In [8]:
sql = """
    CREATE VIEW RecipeWithYear AS
        SELECT *, strftime('%Y', submitted) as year
        FROM Recipe
"""

In [9]:
cur.execute(sql)
con.commit()

In [19]:
pd.read_sql_query("""
    SELECT count(id), year
    FROM RecipeWithYear
    GROUP BY year
""", con)

,count(id),year
0,275,1999
1,104,2000
2,589,2001
3,2644,2002
4,2334,2003
5,2153,2004
6,3130,2005
7,3473,2006
8,4429,2007
9,4029,2008


<p class="task" id="6"></p>

6\. Напишите запрос на языке SQL, который возвращает все строки из таблицы `Recipe` с дополнительным столбцом, содержащем номер рецепта. Рецепты нумеруются целыми числами, начиная с 1, в __рамках каждого года__ в порядке их добавления в БД (столбец `submitted`). Получите результат в виде `pd.DataFrame`. Посчитайте и выведите на экран количество строк полученного `pd.DataFrame`, для которых сгенерированный номер кратен 50.

In [27]:
pd.read_sql_query("""
    WITH RecipeCTE AS (
        SELECT *,
            ROW_NUMBER() OVER(PARTITION BY strftime('%Y', submitted) ORDER BY submitted) as number
    FROM Recipe
    )
    SELECT *
    FROM (RecipeCTE)
""", con)

,id,name,minutes,submitted,description,n_ingredients,number
0,203,chinese plum sauce,115,1999-08-06,chinese plum sauce serve this with egg rolls ...,12.0,1
1,653,b c cherry and raspberry preserves,215,1999-08-09,None,4.0,2
2,360,baked zucchini frittatas,67,1999-08-09,None,NaN,3
3,658,dried fruit roll ups,1495,1999-08-09,fruit roll-ups,4.0,4
4,1144,steak tomato basil pasta,0,1999-08-09,None,11.0,5
...,...,...,...,...,...,...,...
29995,536547,cauliflower ceviche,45,2018-07-30,a healthy ceviche - a perfect appetizer for pa...,8.0,20
29996,536610,miracle home made puff pastry,35,2018-07-31,puff pastry that you can make in minutes? at h...,NaN,21
29997,536729,creole watermelon feta salad,10,2018-08-11,spicy watermelon salad. from tony chachere's s...,NaN,22
29998,536728,gluten free vegemite,2,2018-08-11,gluten free vegemite-like stuff.,3.0,23


<p class="task" id="7"></p>

7\. Используя обобщенное табличное выражение и решение задачи 6, напишите запрос на языке SQL, который вернет количество строк, для которых сгенерированный номер кратен 50. Выполните запрос и выведите количество таких строк на экран.

In [29]:
pd.read_sql_query("""
    WITH RecipeCTE AS (
        SELECT *,
            ROW_NUMBER() OVER(PARTITION BY strftime('%Y', submitted) ORDER BY submitted) as number
    FROM Recipe
    )
    SELECT count(number)
    FROM (RecipeCTE)
    WHERE (number % 50)=0
""", con)

,count(number)
0,589


In [33]:
cur.close()
con.close()